### 1. Configuraçoes Iniciais

#### Imports

In [2]:
import requests
import pandas as pd
import numpy as np
import os

#### Importando dados do funds explorer

In [3]:
url = 'https://www.fundsexplorer.com.br/ranking'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36'' (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    df = pd.read_html(response.content, encoding='utf-8')[0]

#### Exibindo os dados

In [4]:
# df.sort_values('Código do fundo', inplace=True) #classifica os dados utilizando o código do fundo

# display(df) #imprime o DF
# df.info() # exibe o tipo de dados de cada coluna
# df.describe(include='all')
# df['Setor'].unique() #dados univos da coluna de setor
# df.columns


#### Importando dados dos fundos a serem lidos

### 2. Limpeza e Formatação de Dados

In [5]:
#df.isna().sum() #mostra os dados NA

#### Dados categóricos

In [6]:
categorical_columns = ['Código do fundo','Setor'] #cria uma variavel para as colunas categoricas
idx = df[df['Setor'].isna()].index #mostra quais linhas possuem dados NA
#print(df[categorical_columns].isna().sum()) #mostra a soma de todas as linhas com NA
df.drop(idx, inplace=True) #apaga estas linhas
#df[categorical_columns].isna().sum() #mostra a soma de todas as linhas com NA
df[categorical_columns] = df[categorical_columns].astype('category') # define as colunas ocmo categoricas
#df.info()

#### Dados Numéricos

In [7]:
col_floats = list(df.iloc[:,2:-1].columns) #Seleciona da terceira até a penultima linha 
#print(col_floats)
df[col_floats] = df[col_floats].fillna(value=0) #substitui os dados NA por zero
df[col_floats] #valida se existe ainda dados nulos
df[col_floats] = df[col_floats].applymap(lambda x: str(x).replace('R$', '').replace('.0','').replace('.','').replace('%','').replace(',','.')) #limpa os dados removendo caracteres inuteis
df[col_floats] = df[col_floats].astype('float') #define as colunas como float
#df.info()
#df.describe()

##### Dados de P/VPA tem atributos infinitos e está em uma escala diferente

In [8]:
df = df.replace([np.inf, -np.inf], np.nan)  # Converter infinitos para NaN
df = df.dropna() #apaga os dados NA
df['P/VPA'] = df['P/VPA']/100 #convertendo P/VPA para
#df.describe()
#df

### 3. Analises (DF)

In [9]:
indicadores = ['Código do fundo','Setor', 'DY (12M) Acumulado', 'Vacância Física', 'Vacância Financeira', 'P/VPA', 'Quantidade Ativos', 'Liquidez Diária']
indicadoresf = ['DY (12M) Acumulado', 'Vacância Física', 'Vacância Financeira', 'P/VPA', 'Quantidade Ativos', 'Liquidez Diária']

#### Média por Setor

In [33]:
#df.columns #exibe o nome de cabeçalho de todas as colunas do data set DF

In [11]:
df_aux = df[indicadores] #filtra o df pelos  cabeçalhos
media_setor = df_aux.groupby('Setor')[indicadoresf].agg(['mean']) #calcula a media dos indicadores agrupado por setor
media_setoru = media_setor.loc['Hotel', ('P/VPA', 'mean')]

#### Criando uma função com uma estratégia para oportunidades do mercado

In [17]:
def oportunidade_media_setor(df, setor, label_setor='Setor'):
    
    media_setor = df.groupby('Setor')[indicadoresf].agg(['mean']) #calcula a média dos indicadores do setor escolhido
    
    df_setor = df[df[label_setor].isin([setor])]
    
    filter_ = \
            (df_setor['Quantidade Ativos'] > 5) &\
            (df_setor['Liquidez Diária'] > 5000) &\
            (df_setor['P/VPA'] < 1.0) &\
            (df_setor['DY (12M) Acumulado'] > media_setor.loc[setor, ('DY (12M) Acumulado','mean')]) 
            
    print('média do setor Yield: {}'.format(media_setor.loc[setor, ('DY (12M) Acumulado','mean')]))
    print('média do setor p/VPA: {}'.format(media_setor.loc[setor, ('P/VPA','mean')]))
    print('média do setor Ativos: {}'.format(media_setor.loc[setor, ('Quantidade Ativos','mean')]))
    
    return df_setor[filter_]

In [ ]:
list(df['Setor'].unique()) #mostra os setores do filtro selecionado

In [37]:
dataframe = df_aux
setorF = 'Híbrido'
oportunidade = oportunidade_media_setor(dataframe, setorF)
oportunidade.sort_values('DY (12M) Acumulado', ascending=False, inplace=True)
oportunidade

média do setor Yield: 9.32433962264151
média do setor p/VPA: 0.7426415094339622
média do setor Ativos: 3.4150943396226414


,Código do fundo,Setor,DY (12M) Acumulado,Vacância Física,Vacância Financeira,P/VPA,Quantidade Ativos,Liquidez Diária
174,RZTR11,Híbrido,13.39,0.0,0.0,0.91,10,25967.0
272,RECT11,Híbrido,11.11,15.1,0.0,0.60,8,13320.0
200,GALG11,Híbrido,10.69,0.0,0.0,0.96,6,65542.0
267,BRCR11,Híbrido,9.38,21.0,25.8,0.57,15,30190.0


### 4. Analises (FIIs Pessoais)

In [22]:
os.chdir("C:/Users/luis-/projetos/Python/analise-acoes/") #selecionando o diretorio do arquivo CSV a ser lido
tickets = pd.read_csv("./tickets.csv", sep = ',') #importando dados de um csv utilizando o pandas
tickets = tickets['Ativo'].tolist() #convertendo o data frame para lista

In [24]:
filtro = df.iloc[:, 0].isin(tickets) #filtando os dados utilizando as acoes do arquivo csv
df_filtro = df.loc[filtro]
#display(df_filtro)

In [25]:
df_auxp = df_filtro[indicadores] #filtra o df pelos  cabeçalhos
#display(df_auxp)

In [30]:
allsetors = list(df_filtro['Setor'].unique())

In [32]:
oportunidade = oportunidade_media_setor(df_aux, setor='Hotel')
oportunidade.sort_values('DY (12M) Acumulado', ascending=False, inplace=True)
oportunidade

média do setor Yield: 11.786666666666667
média do setor p/VPA: 0.8766666666666666
média do setor Ativos: 9.333333333333334


,Código do fundo,Setor,DY (12M) Acumulado,Vacância Física,Vacância Financeira,P/VPA,Quantidade Ativos,Liquidez Diária
